# Installations

In [1]:
! pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.1 MB/s eta 0:00:00


In [ ]:
# ! pip install -q kaggle
# ! pip install -q kagglehub

In [ ]:
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# Method 1 (Part 1): CLI (download zip file)
# ! kaggle datasets download salmahazemm/tennis-analysis

In [ ]:
# ! rm -rf "/content/drive/MyDrive/datasetyolo"

In [2]:
# Method 1 (Part 2): Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Method 1 (Part 3)
# ! unzip -q "/content/tennis-analysis.zip" -d "/content/drive/MyDrive/datasetyolo"

In [ ]:
# # Method 2
# import kagglehub
# # Download latest version of dataset
# path = kagglehub.dataset_download("salmahazemm/tennis-analysis")
# print("Path to dataset files:", path)

In [4]:
! cd /content/drive/MyDrive/datasetyolo/

In [6]:
! pwd

/content/drive/MyDrive/datasetyolo


# Training

#### You will need to choose the L4 GPU

In [7]:
! nvidia-smi

Thu Jan 15 15:40:56 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

#### '/root/.config/Ultralytics/settings.json'

In [ ]:
# Note: L4 supports yolo8x, but not yolo11x
! yolo task=detect mode=train model=yolov8x.pt data=/content/drive/MyDrive/datasetyolo/data.yaml epochs=100 imgsz=800 plots=True


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/datasetyolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=800, int8=False

# Single Image Testing

In [8]:
! cd /content/

In [9]:
from ultralytics import YOLO
import cv2
import time
from PIL import Image, ImageDraw
import numpy as np
import datetime

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Link to Best weights: https://drive.google.com/file/d/1nI8n3Q6yn7c_X6a_iB15ZlUvqq-aWyd_/view?usp=drive_link

In [10]:
yolo_model = YOLO("/content/drive/MyDrive/datasetyolo/runs/detect/train4/weights/best_instructor.pt")

In [11]:
img = "/content/drive/MyDrive/datasetyolo/test/images/synframe525_jpg.rf.80de3c817568c67ba6cda200b7c04316.jpg"
results = yolo_model (img, conf=0.01)
results_for_image = results[0]
num_detections = len(results_for_image.boxes)
results_for_image.save (filename="results.png")
print(f"Number of objects detected in this image: {num_detections}")



image 1/1 /content/drive/MyDrive/datasetyolo/test/images/synframe525_jpg.rf.80de3c817568c67ba6cda200b7c04316.jpg: 480x800 2 tennis balls, 58.9ms
Speed: 13.4ms preprocess, 58.9ms inference, 38.5ms postprocess per image at shape (1, 3, 480, 800)
Number of objects detected in this image: 2


In [12]:
# Update to select the object with max confidence.

for i, detection in enumerate(results_for_image.boxes):
  # print(detection) # verbose
  confidence = detection.conf.item()
  bbox = detection.xyxy[0].cpu().numpy()
  print(bbox, confidence)
  # break

[     528.35      255.53      541.53      269.94] 0.7364024519920349
[     528.43      253.22       544.7       270.3] 0.03982800245285034


# Full Video Testing

In [47]:
! pip install -q sahi

In [54]:
from sahi import AutoDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction, predict
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url


In [69]:
# Initializations
SAHI = False
confidence = 0.01

There are two versions of the ball detection function: STANDARD and SAHI

Define only one of these two functions.

In [70]:
# STANDARD VERSION

def ball_detections (image):
  results = yolo_model (image, conf=confidence)
  results_for_image = results[0]
  num_detections = len(results_for_image.boxes)
  bbox = None
  if num_detections > 0:
    for i, detection in enumerate(results_for_image.boxes):
      bbox = detection.xyxy[0].cpu().numpy()
      # break
  return num_detections, bbox


In [71]:
# SAHI VERSION
# SAHI = True

# To perform sliced prediction we need to specify slice parameters.
# We will perform prediction over slices of 256x256 with an overlap ratio of 0.2:
# Video frame size: frame width = 1920, frame height = 1080

# def get_results (image):
#   result = get_sliced_prediction (
#     image,
#     yolo_model,
#     slice_height = 256,
#     slice_width = 256,
#     overlap_height_ratio = 0.2,
#     overlap_width_ratio = 0.2,
# )

# def ball_detections (image):
#   # results = yolo_model (image, conf=confidence)
#   results = get_results (image)
#   print (results)
#   # results_for_image = results[0]
#   # num_detections = len(results_for_image.boxes)
#   # bbox = None
#   # if num_detections > 0:
#   #   for i, detection in enumerate(results_for_image.boxes):
#   #     bbox = detection.xyxy[0].cpu().numpy()
#   #     # break
#   # return num_detections, bbox
#   return 0, None


In [73]:
# Initialize the video run
import cv2
import datetime
from ultralytics import YOLO

weights = "best_instructor" # "best_bob"
# dataset = "train" # "test"
confidence = 0.01

if not SAHI:
  yolo_model = YOLO (f"/content/drive/MyDrive/datasetyolo/weights/{weights}.pt")
# else:
#   yolo_model = AutoDetectionModel.from_pretrained (
#     model_type = "ultralytics",  # Specify the framework type
#     # Path to your custom weights
#     model_path = f"/content/drive/MyDrive/datasetyolo/weights/{weights}.pt",
#     confidence_threshold = confidence,
#     device = "cuda:0", # "cpu"
# )
input_video_path  = f"/content/drive/MyDrive/tennis-videos/tennis_video.mp4"
output_video_path = f"/content/drive/MyDrive/tennis-videos/output_video_{weights}.mp4"

video_cap = cv2.VideoCapture (input_video_path)
frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(video_cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(video_cap.get(cv2.CAP_PROP_FPS))
print(f"total_frames = {total_frames}")
print(f"fps = {fps}")
print(f"frame width = {frame_width}")
print(f"frame height = {frame_height}")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# fps = 1  # Slow the output video down for easier analysis
writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))


total_frames = 214
fps = 30
frame width = 1920
frame height = 1080


In [74]:
# Loop over the video frames
while video_cap.isOpened():

    start = datetime.datetime.now()
    ret, frame = video_cap.read()
    if not ret: break
    num_detections, ball_bbox = ball_detections (frame) # call the trained model

    # if ball_bbox is not None:
    if num_detections > 0:
      xmin_ball = int(ball_bbox[0])
      ymin_ball = int(ball_bbox[1])
      xmax_ball = int(ball_bbox[2])
      ymax_ball = int(ball_bbox[3])
      cv2.rectangle (frame, (xmin_ball, ymin_ball), (xmax_ball, ymax_ball), (255, 0, 0), 2)
      cv2.putText (frame, "ball", (xmin_ball, ymin_ball - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
      cv2.putText (frame, f"MODEL WEIGHTS: {weights}", (300, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
      cv2.putText (frame, f"CONFIDENCE LEVEL: {confidence}", (300, 325), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
      cv2.putText (frame, f"BALLS FOUND: {num_detections}", (300, 350), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
    else:
      cv2.putText (frame, "NO BALLS FOUND", (500, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    writer.write(frame)
    end = datetime.datetime.now()
    print(f"Time to process 1 frame: {(end - start).total_seconds() * 1000:.0f} miliseconds")

video_cap.release()
writer.release()
# cv2.destroyAllWindows()



0: 480x800 8 tennis balls, 88.1ms
Speed: 3.9ms preprocess, 88.1ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 800)
Time to process 1 frame: 839 miliseconds

0: 480x800 8 tennis balls, 48.3ms
Speed: 4.1ms preprocess, 48.3ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 800)
Time to process 1 frame: 78 miliseconds

0: 480x800 7 tennis balls, 49.8ms
Speed: 4.4ms preprocess, 49.8ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 800)
Time to process 1 frame: 77 miliseconds

0: 480x800 8 tennis balls, 51.8ms
Speed: 4.4ms preprocess, 51.8ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 800)
Time to process 1 frame: 83 miliseconds

0: 480x800 7 tennis balls, 47.4ms
Speed: 5.8ms preprocess, 47.4ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 800)
Time to process 1 frame: 78 miliseconds

0: 480x800 8 tennis balls, 52.3ms
Speed: 4.4ms preprocess, 52.3ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 800)
Time to

# OPTIONAL SECTION
Create a video from the dataset files.

In [ ]:
def create_video_from_directory_of_images (dataset_folder = "test", video_name = "video_1"):

    # Imports
    import cv2
    import glob
    import os

    # --- Configuration ---
    image_folder = f"/content/drive/MyDrive/datasetyolo/{dataset_folder}/images"
    video_name = f"/content/drive/MyDrive/tennis-videos/{video_name}_{dataset_folder}.mp4"
    fps = 5 # Frames per second for the output video
    # --- End Configuration ---

    # 1. Get and sort the list of images
    # Use glob to find all PNG or JPG images and sort them for correct sequence
    images = sorted(glob.glob(os.path.join(image_folder, '*.png')) + glob.glob(os.path.join(image_folder, '*.jpg')))
    if not images:
        print(f"No images found in {image_folder}")
        return

    # 2. Read the first image to get dimensions
    # All images in the output video must have the same size
    frame = cv2.imread(images[0])
    height, width, layers = frame.shape
    size = (width, height)
    print(f"Number of images = {len(images)}")

    # 3. Define the codec and create VideoWriter object
    # For .avi files, 'MJPG' is a common and safe choice.
    # For .mp4 files, 'mp4v' can be used.
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(video_name, fourcc, fps, size)

    # 4. Loop over the image files
    for image_path in images:
        img = cv2.imread(image_path)
        # Optional: Resize image if dimensions can vary
        img = cv2.resize(img, size)
        video.write(img)

    # 5. Release the video writer object
    video.release()
    # cv2.destroyAllWindows()
    print(f"Video '{video_name}' created successfully from {len(images)} images.")
    return


In [ ]:
# Call the above function
create_video_from_directory_of_images (dataset_folder = "train", video_name = "input_video")


Number of images = 428
Video '/content/drive/MyDrive/tennis-videos/input_video_train.mp4' created successfully from 428 images.
